<a href="https://colab.research.google.com/github/shanisheashua/final-project---group-13/blob/main/calendar_by_syllabus.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget https://raw.githubusercontent.com/shanisheashua/final-project---group-13/main/training_physics.xlsx -O training_physics.xlsx
!wget https://raw.githubusercontent.com/shanisheashua/final-project---group-13/main/training_python.xlsx -O training_python.xlsx
!pip install icalendar
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import matplotlib.pyplot as plt
from datetime import timedelta, datetime

# Step 1: Data Loading and Preparation
def user_input():
    chosen_subject = int(input(f"\nPlease choose a subject:\n   1 for python \n   2 for physics\n"))
    if chosen_subject == 1:
        subjects = ['Data Structures', 'Recursion', 'OOP', 'Pandas', 'NumPy']
        file_path = "training_python.xlsx"
    else:
        subjects = ['Forces', 'Circular Motion', 'Work and Energy', 'Momentum and Impulse', 'Harmonic motion']
        file_path = "training_physics.xlsx"

    data = pd.read_excel(file_path)

    # Create empty lists for each criterion
    topic_weights_list = []
    difficulties_list = []
    independence_list = []
    previous_knowledge_list = []

    # Ask the user to input criteria for each subject
    for subject in subjects:
        print(f"\nEnter the details for subject: {subject}")

        # Input for each criterion (ensuring values are between 1 and 5)
        while True:
            try:
                topic_weight = int(input(f"  Topic Weight/Importance (1-5) for {subject}: "))
                if topic_weight < 1 or topic_weight > 5:
                    print("Please enter a value between 1 and 5.")
                    continue
                break
            except ValueError:
                print("Invalid input. Please enter a number between 1 and 5.")

        while True:
            try:
                difficulty = int(input(f"  Difficulty (1-5) for {subject}: "))
                if difficulty < 1 or difficulty > 5:
                    print("Please enter a value between 1 and 5.")
                    continue
                break
            except ValueError:
                print("Invalid input. Please enter a number between 1 and 5.")

        while True:
            try:
                independence = int(input(f"  Independence (1-5) for {subject}: "))
                if independence < 1 or independence > 5:
                    print("Please enter a value between 1 and 5.")
                    continue
                break
            except ValueError:
                print("Invalid input. Please enter a number between 1 and 5.")

        while True:
            try:
                prev_knowledge = int(input(f"  Previous Knowledge (1-5) for {subject}: "))
                if prev_knowledge < 1 or prev_knowledge > 5:
                    print("Please enter a value between 1 and 5.")
                    continue
                break
            except ValueError:
                print("Invalid input. Please enter a number between 1 and 5.")

        # Append the inputs to the corresponding lists
        topic_weights_list.append(topic_weight)
        difficulties_list.append(difficulty)
        independence_list.append(independence)
        previous_knowledge_list.append(prev_knowledge)

    # Now ask for the number of days
    while True:
        try:
            days = int(input("\nEnter the number of days: "))
            if days < 1:
                print("Please enter a positive number of days.")
                continue
            break
        except ValueError:
            print("Invalid input. Please enter a number.")

    new_student_input = pd.DataFrame({
        'Topic': subjects,
        'Topic Weight/Importance (1-5)': topic_weights_list,
        'Difficulty (1-5)': difficulties_list,
        'Independence (1-5)': independence_list,
        'Previous Knowledge (1-5)': previous_knowledge_list
    })

    return new_student_input, days, data


# Step 2: Data Preprocessing
def preprocess_data(data):
    # Ensure the columns have correct data types
    data['Topic Weight/Importance (1-5)'] = data['Topic Weight/Importance (1-5)'].astype(int)
    data['Difficulty (1-5)'] = data['Difficulty (1-5)'].astype(int)
    data['Independence (1-5)'] = data['Independence (1-5)'].astype(int)
    data['Previous Knowledge (1-5)'] = data['Previous Knowledge (1-5)'].astype(int)

    # Convert 'Topic' to a categorical feature
    data['Topic'] = data['Topic'].astype(str)

    return data

# Step 3: Model Training
def train_model(data):
    # Features and target variable
    X = data[['Topic', 'Topic Weight/Importance (1-5)', 'Difficulty (1-5)', 'Independence (1-5)',
              'Previous Knowledge (1-5)']]
    y = data['Suggested Hours']

    # Define column transformer (for encoding 'Topic' and scaling the numerical features)
    preprocessor = ColumnTransformer(
        transformers=[
            ('topic', OneHotEncoder(), ['Topic']),  # One-hot encode the 'Topic'
            ('num', StandardScaler(), ['Topic Weight/Importance (1-5)', 'Difficulty (1-5)',
                                       'Independence (1-5)', 'Previous Knowledge (1-5)'])  # Scale numerical features
        ])

    # Create a pipeline with a preprocessor and a Random Forest Regressor
    model_pipeline = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('model', RandomForestRegressor(n_estimators=100, random_state=42))
    ])

    # Train-test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Fit the model
    model_pipeline.fit(X_train, y_train)

    return model_pipeline

# Step 4: Predict for New Student
def predict_study_hours(model_pipeline, new_student_data):
    # Predict the study hours for a new student based on their input
    predicted_hours = model_pipeline.predict(new_student_data)
    return predicted_hours

# Step 4: create schedule
def create_schedule(percentage_hours):
    # Distribute study hours for each topic across the days
    schedule = {day: [] for day in range(1, ddays + 1)}

    # Remaining hours for each topic
    remaining_hours = {item[0]: item[1] for item in percentage_hours}

    # Create a list of topics to cycle through to prevent consecutive long study periods
    topic_queue = list(remaining_hours.keys())

    for day in range(1, ddays + 1):
        current_time = start_time
        while current_time < end_time:
            # Skip the break
            if break_start <= current_time < break_end:
                current_time = break_end
                continue

            # Find the next topic to study
            for _ in range(len(topic_queue)):
                topic = topic_queue.pop(0)  # Rotate topics to avoid consecutive long sessions
                hours_left = remaining_hours[topic]
                if hours_left > 0:
                    # Study for up to 3 hours or until the topic is done
                    study_duration = min(3, hours_left, (end_time - current_time).seconds / 3600)

                    # Ensure study session does not overlap with the break
                    study_end_time = current_time + timedelta(hours=study_duration)
                    if study_end_time > break_start and current_time < break_start:
                        study_duration = (break_start - current_time).seconds / 3600

                    # Ensure study session does not exceed the end time
                    if study_end_time > end_time:
                        study_duration = (end_time - current_time).seconds / 3600

                    # Update the schedule
                    if study_duration > 0:
                        schedule[day].append((current_time.strftime("%H:%M"),
                                              (current_time + timedelta(hours=study_duration)).strftime("%H:%M"),
                                              topic))

                        # Deduct studied hours
                        remaining_hours[topic] -= study_duration
                        current_time += timedelta(hours=study_duration)

                        # Re-add the topic to the queue if it still has hours left
                        if remaining_hours[topic] > 0:
                            topic_queue.append(topic)

                    break  # Move to the next time slot

            # If no topic has hours left, end the day early
            if all(hours <= 0 for hours in remaining_hours.values()):
                break

    return schedule

# Main Code Execution
if __name__ == "__main__":
    # Example: New student input (modify this based on your input structure)
    new_student_input, ddays, data = user_input()

    # Preprocess the data
    data = preprocess_data(data)

    # Train the model
    model_pipeline = train_model(data)

    # Predict the study hours for the new student
    predicted_hours = predict_study_hours(model_pipeline, new_student_input)

    percentage_hours = []
    print(f"\n_____________________________\n")
    # Output the suggested study hours
    for idx, hours in enumerate(predicted_hours):
        percentage_hours.append([new_student_input['Topic'].iloc[idx], round(hours), (hours/sum(predicted_hours))])
        print(f"Suggested hours for topic {new_student_input['Topic'].iloc[idx]}: {hours:.2f}")

    # Time constraints
    start_time = datetime.strptime("09:00", "%H:%M")
    end_time = datetime.strptime("19:00", "%H:%M")
    break_start = datetime.strptime("12:00", "%H:%M")
    break_end = datetime.strptime("13:00", "%H:%M")

    break_hours = 1
    study_time = 10

    # Calculate total available hours per day
    total_hours_per_day = (end_time - start_time).seconds / 3600 - break_hours

    # Check the condition
    total_hours = ddays * (total_hours_per_day)
    if total_hours >= sum(predicted_hours):
        schedule = create_schedule(percentage_hours)

    else:
        for row in percentage_hours:
            row[1] = round(row[2] * total_hours)
        schedule = create_schedule(percentage_hours)

    from icalendar import Calendar, Event

    # Set the start date (tomorrow)
    start_date = datetime.today() + timedelta(days=1)

    # Create the calendar
    cal = Calendar()

    # Add events to the calendar
    for day, events in schedule.items():
        day_date = start_date + timedelta(days=day-1)  # Calculate the date for each day
        for start, end, topic in events:
            event = Event()
            event.add('summary', topic)
            event.add('dtstart', day_date.replace(hour=int(start[:2]), minute=int(start[3:5]), second=int('00')))
            event.add('dtend', day_date.replace(hour=int(end[:2]), minute=int(end[3:5]), second=int('00')))
            event.add('dtstamp', datetime.now())
            cal.add_component(event)

    # Save the calendar to an .ics file
    with open('study_schedule.ics', 'wb') as f:
        f.write(cal.to_ical())

    print("\nCalendar file created: study_schedule.ics\n\nGOOD LUCK! :)")

--2025-04-02 10:38:46--  https://raw.githubusercontent.com/shanisheashua/final-project---group-13/main/training_physics.xlsx
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11511 (11K) [application/octet-stream]
Saving to: ‘training_physics.xlsx’

training_physics.xl 100%[===================>]  11.24K  --.-KB/s    in 0.001s  

2025-04-02 10:38:47 (17.6 MB/s) - ‘training_physics.xlsx’ saved [11511/11511]

--2025-04-02 10:38:47--  https://raw.githubusercontent.com/shanisheashua/final-project---group-13/main/training_python.xlsx
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP reques